In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.linear_model import LogisticRegression



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving credit_risk_dataset.csv to credit_risk_dataset.csv


In [ ]:
data = pd.read_csv('credit_risk_dataset.csv')
# data.head()



# Restructure dataframe 

---

Append target to end of dataframe 



In [ ]:
cols = data.columns.tolist()

In [ ]:
target = cols[8]

In [ ]:
cols = cols[:8] + cols[9:]
cols.append(target)
data = data[cols]
data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,0.59,Y,3,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3,1
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2,1
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4,1


# Clean Data

---

Remove null and outliers 



In [ ]:

data.isnull().sum()


person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
loan_status                      0
dtype: int64

In [ ]:
cleanData = data.dropna(axis=0)

In [ ]:
fig = px.scatter_matrix(cleanData,
                        dimensions=["person_age", "person_income", "person_emp_length", "loan_amnt", "loan_int_rate"],
                        labels={col: col.replace('_', ' ') for col in cleanData.columns},
                        height=900, color="loan_status",

                        color_continuous_scale=px.colors.diverging.Portland)
fig

In [ ]:
cleanData = cleanData[cleanData["person_age"] <= 100]
cleanData = cleanData[cleanData["person_emp_length"] <= 100]
cleanData = cleanData[cleanData["person_income"] <= 4000000]

# Data Visualization


In [ ]:
#Categorical Diagram
fig = px.parallel_categories(cleanData,
                             color_continuous_scale=px.colors.sequential.Aggrnyl,
                             color="loan_status",
                             dimensions=['person_home_ownership', 'loan_intent', "loan_grade", 'cb_person_default_on_file'],
                             labels={col:col.replace('_', ' ') for col in cleanData.columns})
fig.show()

# Categorical Encoding

In [ ]:
#One hot encoding of categorical variables
cleanData = pd.get_dummies(data=cleanData,columns=['person_home_ownership','loan_intent','loan_grade','cb_person_default_on_file'])
cleanData

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,...,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y
1,21,9600,5.0,1000,11.14,0.10,2,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,25,9600,1.0,5500,12.87,0.57,3,1,1,0,...,0,0,0,1,0,0,0,0,1,0
3,23,65500,4.0,35000,15.23,0.53,2,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,24,54400,8.0,35000,14.27,0.55,4,1,0,0,...,0,0,0,1,0,0,0,0,0,1
5,21,9900,2.0,2500,7.14,0.25,2,1,0,0,...,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,1.0,5800,13.16,0.11,30,0,1,0,...,0,0,0,1,0,0,0,0,1,0
32577,54,120000,4.0,17625,7.49,0.15,19,0,1,0,...,0,1,0,0,0,0,0,0,1,0
32578,65,76000,3.0,35000,10.99,0.46,28,1,0,0,...,0,0,1,0,0,0,0,0,1,0
32579,56,150000,5.0,15000,11.48,0.10,26,0,1,0,...,0,0,1,0,0,0,0,0,1,0


# Data Split

In [ ]:
X = cleanData.drop('loan_status',axis=1)
y = cleanData['loan_status']

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.30, random_state=20)

# XGBoost 


In [ ]:
train = xgb.DMatrix(Xtrain, label=ytrain)
test = xgb.DMatrix(Xtest, label=ytest)


In [ ]:
param = {
    'max_depth': 4, 
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 2
}

epochs = 10

In [ ]:
model = xgb.train(param, train, epochs)

In [ ]:
predictions = model.predict(test)

# Model Evaluation

In [ ]:
cm = np.array(confusion_matrix(ytest, predictions, labels = [1,0]))
confusion = pd.DataFrame(cm, index = ['Default', 'Non-Default'],
                        columns = ['Predict Default', 'Predict Non-Default'])
confusion

,Predict Default,Predict Non-Default
Default,1327,541
Non-Default,62,6660


In [ ]:
print(classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96      6722
           1       0.96      0.71      0.81      1868

    accuracy                           0.93      8590
   macro avg       0.94      0.85      0.89      8590
weighted avg       0.93      0.93      0.93      8590



# SMOTE Oversampling

In [ ]:
X.value_counts()
y.value_counts()

0    22430
1     6202
Name: loan_status, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE 

In [ ]:
smote = SMOTE(sampling_strategy='minority')

In [ ]:
X_sm, y_sm = smote.fit_resample(X, y)

In [ ]:
X_sm.value_counts()
y_sm.value_counts()

0    22430
1    22430
Name: loan_status, dtype: int64

In [ ]:
Xtrain_sm, Xtest_sm, ytrain_sm, ytest_sm = train_test_split(X_sm, y_sm, test_size=.30, random_state=20)

# XGBoost - Balanced SMOTE Data

In [ ]:
train_sm = xgb.DMatrix(Xtrain_sm, label=ytrain_sm)
test_sm = xgb.DMatrix(Xtest_sm, label=ytest_sm)

# Hyperparameters 
param = {
    'max_depth': 4, 
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 2
}

epochs = 10

In [ ]:
model = xgb.train(param, train_sm, epochs)

In [ ]:
predictions = model.predict(test_sm)

# SMOTE Evaluation

In [ ]:
cm = np.array(confusion_matrix(ytest_sm, predictions, labels = [1,0]))
confusion = pd.DataFrame(cm, index = ['Default', 'Non-Default'],
                        columns = ['Predict Default', 'Predict Non-Default'])
confusion

,Predict Default,Predict Non-Default
Default,6047,680
Non-Default,290,6441


In [ ]:
print(classification_report(ytest_sm, predictions))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      6731
           1       0.95      0.90      0.93      6727

    accuracy                           0.93     13458
   macro avg       0.93      0.93      0.93     13458
weighted avg       0.93      0.93      0.93     13458

